In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from syllables import estimate

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [58]:
def get_text(i, url):
      
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.find('title').text
    if i in {0,14,54,55,63,70,71,75}:
        content = soup.find_all('div', class_='tdb-block-inner td-fix-index')
        content = content[14]
    elif i in {7, 20, 107}:
        return ''
    else:    
        content = soup.find('div', class_="td-post-content tagdiv-type")
    

    result = ''
  

    if content:
        paragraphs = content.find_all('p')  # Use find_all() to get all <p> tags
        for paragraph in paragraphs:
            paragraph_text = paragraph.get_text()  # Using strip=True to remove whitespace strip=True        
            result = result + " " + paragraph_text

    else:
        print("No article content found.", i)

    return  title + ' ' + result 

In [4]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [63]:
def preprocess(text):
    
    text = text.lower()
    text = re.sub('[^a-zA-Z]+',' ', text).strip() 
    tokens = word_tokenize(text)    

    #lemmatize and remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [64]:
# Read the positive and negative word lists
with open('positive-words.txt', 'r') as f:
    positive_words = set(f.read().splitlines())

with open('negative-words.txt', 'r') as f:
    negative_words = set(f.read().splitlines())

In [73]:
def pos_neg(idx, url, tokens):
        
    # Count positive and negative words
    positive_count = sum(1 for word in tokens if lemmatizer.lemmatize(word) in positive_words)
    negative_count = sum(1 for word in tokens if lemmatizer.lemmatize(word) in negative_words)
    #polarity score
    polarity_score =  (positive_count - negative_count) / ((positive_count + negative_count) + 0.000001)
    #subjective score
    total_words = len(tokens)
    subjective_score = (positive_count + negative_count)/ ((total_words) + 0.000001)
    # average sentence length
    number_of_sentences = sent_tokenize(get_text(idx, url))
    number_of_sentence = len(number_of_sentences) - 1 
    avg_sent_len =  total_words / number_of_sentence  
    #complex words
    complex_words = len([word for word in tokens if estimate(word) > 2])
    percentage_complex = complex_words / (total_words + 0.0001)
    percentage_complex
    #fog index
    fog_index = 0.4 * (avg_sent_len + percentage_complex)
    #personal pronouns
    personal_pronouns = {"I", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them", "myself", "yourself", "himself", "herself", "itself", "ourselves", "yourselves", "themselves"}
    personal_pronoun_count = sum(1 for word in tokens if word in personal_pronouns)
    #avg word len
    total_characters = sum(len(word) for word in tokens)
    total_words = len(tokens)
    average_word_length = total_characters / total_words if total_words > 0 else 0


    return [positive_count, negative_count, polarity_score, subjective_score, avg_sent_len, percentage_complex, fog_index, avg_sent_len, complex_words, total_words, complex_words, personal_pronoun_count, average_word_length] 



writing output 

In [6]:
df = pd.read_csv('Output.csv')

In [74]:
for idx in df.index:
    url = df['URL'][idx]
    text = get_text(idx, url)
    tokens = preprocess(text)
    data = pos_neg(idx, url, tokens)
    df.iloc[idx : idx + 1, 2:] = data
    
   


In [75]:
df.to_csv('final.csv')